<a href="https://colab.research.google.com/github/Imeneallouche/Amana/blob/main/AMANA_Urgency_Tier_Prediction_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **AMANA Urgency Tier Prediction Model**
- AI-powered urgency classification for charitable missions
- Combines deadline proximity, beneficiary vulnerability, and historical impact data

## 1. Environment Setup

1.   Élément de liste
2.   Élément de liste


Import required libraries and configure settings

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import joblib

In [ ]:
# Configuration
np.random.seed(42)
urgency_weights = {
    'deadline': 0.4,
    'vulnerability': 0.4,
    'historical': 0.2
}

## 2. Feature Engineering Pipeline

In [ ]:
class UrgencyCalculator:
    def __init__(self, historical_data_path='historical_missions.csv'):
        self.scaler = MinMaxScaler()
        self.historical_df = pd.read_csv(historical_data_path)
        self.load_historical_patterns()

    def load_historical_patterns(self):
        """Analyze historical mission data patterns"""
        self.historical_urgency = self.historical_df.groupby('category')['urgency'].mean().to_dict()

    def calculate_deadline_score(self, deadline_date):
        """Calculate time sensitivity score (0-1 scale)"""
        days_left = (deadline_date - datetime.now()).days
        return max(0, 1 - days_left/30)  # 30-day max horizon

    def calculate_vulnerability_score(self, beneficiaries):
        """Calculate composite vulnerability score"""
        score = 0
        # Medical needs multiplier
        score += beneficiaries.get('medical_conditions', 0) * 1.2
        # Food insecurity multiplier
        score += beneficiaries.get('food_insecurity_days', 0) * 0.8
        # Age vulnerability (children/elderly)
        score += (beneficiaries.get('children', 0) +
                 beneficiaries.get('elderly', 0)) * 1.0
        return np.log1p(score)  # Logarithmic scaling

    def calculate_historical_score(self, category):
        """Get historical urgency for mission category"""
        return self.historical_df.get(category, 0.5)

    def calculate_urgency(self, mission_data):
        """Main prediction method"""
        # Calculate component scores
        deadline_score = self.calculate_deadline_score(
            mission_data['deadline'])
        vulnerability_score = self.calculate_vulnerability_score(
            mission_data['beneficiaries'])
        historical_score = self.calculate_historical_score(
            mission_data['category'])

        # Combine scores with weights
        composite_score = (
            deadline_score * urgency_weights['deadline'] +
            vulnerability_score * urgency_weights['vulnerability'] +
            historical_score * urgency_weights['historical']
        )

        # Classify urgency tier
        if composite_score >= 0.7:
            return "URGENT"
        elif composite_score >= 0.4:
            return "MODERATE"
        else:
            return "NON-URGENT"

## 3. Model Training
Using historical data to train supplemental ML model

In [ ]:
def train_ml_model():
    """Train machine learning model on historical data"""
    df = pd.read_csv('historical_missions.csv')

    # Feature engineering
    df['days_remaining'] = (pd.to_datetime(df['deadline']) -
                           pd.Timestamp.now()).dt.days
    df['vulnerability_score'] = np.log1p(df['medical_conditions']*1.2 +
                                      df['food_days']*0.8)

    X = df[['days_remaining', 'vulnerability_score', 'category_encoded']]
    y = df['urgency_encoded']

    model = RandomForestClassifier()
    model.fit(X, y)

    joblib.dump(model, 'urgency_classifier.pkl')
    return model

## 4. Production Inference
Example usage with real mission data

In [ ]:
# Sample mission data
mission_example = {
    'deadline': datetime(2024, 4, 15),
    'beneficiaries': {
        'medical_conditions': 5,
        'food_insecurity_days': 14,
        'children': 8,
        'elderly': 3
    },
    'category': 'medical'
}

# Initialize calculator
calculator = UrgencyCalculator()

# Get prediction
urgency_tier = calculator.calculate_urgency(mission_example)
print(f"Predicted Urgency Tier: {urgency_tier}")

## 5. Model Evaluation Metrics
Validation against test dataset

In [ ]:
def evaluate_model():
    test_df = pd.read_csv('test_missions.csv')
    calculator = UrgencyCalculator()

    results = []
    for _, row in test_df.iterrows():
        pred = calculator.calculate_urgency(row)
        results.append(pred == row['true_urgency'])

    accuracy = np.mean(results)
    print(f"Model Accuracy: {accuracy:.2%}")
    return accuracy